# Education Outcomes EDA: Nigeria Literacy Analysis

This notebook analyzes factors associated with literacy outcomes across Nigerian states using synthetic data.

**Objectives:**
1. Generate or load synthetic education data
2. Perform exploratory data analysis (EDA)
3. Identify key patterns and disparities
4. Fit logistic regression models to identify associations
5. Present findings in plain language

**Note:** All data in this analysis is synthetic and generated for demonstration purposes.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.inspection import permutation_importance
import statsmodels.api as sm

# Visualization setup
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

## 1. Data Generation and Loading

First, we'll generate synthetic data or load existing data.

In [ ]:
# Generate data if it doesn't exist
from src.data.generate_synthetic_data import main as generate_data, generate_group_features, generate_individual_data, create_state_summaries

raw_data_path = Path("data/raw/education_outcomes_individual.csv")

if not raw_data_path.exists():
    print("Generating synthetic data...")
    generate_data()
else:
    print(f"Data already exists at {raw_data_path}")

# Load the data
print("\nLoading data...")
df_raw = pd.read_csv(raw_data_path)
print(f"Loaded {len(df_raw):,} rows, {len(df_raw.columns)} columns")
print(f"\nDataset shape: {df_raw.shape}")

## 2. Data Cleaning

Clean and prepare the data for analysis.

In [ ]:
from src.data.cleaning import clean_and_save

# Run cleaning pipeline
clean_and_save(
    raw_path="data/raw/education_outcomes_individual.csv",
    out_dir="data/processed"
)

# Load cleaned data
df = pd.read_parquet("data/processed/education_outcomes_cleaned.parquet")
print(f"\n✓ Cleaned data loaded: {len(df):,} rows")

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Data info
df.info()

## 3. Summary Statistics

Let's examine overall statistics and breakdowns by key groups.

In [ ]:
# Overall literacy rate
overall_literacy = df['literacy_outcome'].mean() * 100
print(f"Overall Literacy Rate: {overall_literacy:.1f}%")

# By location
print("\nLiteracy Rate by Location:")
location_rates = df.groupby('location')['literacy_outcome'].mean() * 100
for loc, rate in location_rates.items():
    print(f"  {loc.capitalize()}: {rate:.1f}%")

# By sex
print("\nLiteracy Rate by Sex:")
sex_rates = df.groupby('sex')['literacy_outcome'].mean() * 100
for sex, rate in sex_rates.items():
    print(f"  {sex}: {rate:.1f}%")

# By state (top and bottom 5)
print("\nTop 5 States by Literacy Rate:")
state_rates = df.groupby('state')['literacy_outcome'].mean().sort_values(ascending=False) * 100
for state, rate in state_rates.head().items():
    print(f"  {state}: {rate:.1f}%")

print("\nBottom 5 States by Literacy Rate:")
for state, rate in state_rates.tail().items():
    print(f"  {state}: {rate:.1f}%")

In [ ]:
# Summary statistics for key features
feature_cols = [
    'enrollment_rate', 'pupil_teacher_ratio', 'teacher_qualification_rate',
    'household_poverty_rate', 'mother_education_years', 'household_size',
    'internet_access_rate', 'textbook_availability_index',
    'travel_time_to_school_min', 'electricity_access_rate'
]

df[feature_cols].describe().round(2)

## 4. Exploratory Data Analysis (EDA)

### 4.1 Feature Distributions

In [ ]:
# Distribution of key features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Distribution of Key Features', fontsize=16, fontweight='bold')

key_features = [
    'enrollment_rate', 'pupil_teacher_ratio', 'teacher_qualification_rate',
    'household_poverty_rate', 'mother_education_years', 'internet_access_rate'
]

for idx, feature in enumerate(key_features):
    row = idx // 3
    col = idx % 3
    axes[row, col].hist(df[feature], bins=50, alpha=0.7, color='steelblue')
    axes[row, col].set_xlabel(feature.replace('_', ' ').title())
    axes[row, col].set_ylabel('Frequency')
    axes[row, col].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('reports/figures/feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: reports/figures/feature_distributions.png")

### 4.2 Correlation Analysis

In [ ]:
from src.viz.plots import plot_correlation_heatmap

# Create correlation heatmap
numeric_features = feature_cols + ['literacy_outcome']
plot_correlation_heatmap(
    df,
    features=numeric_features,
    output_path='reports/figures/correlation_heatmap.png',
    title='Feature Correlation Heatmap'
)

# Display the saved figure
from IPython.display import Image
Image('reports/figures/correlation_heatmap.png')

**Key Observations:**
- Positive correlations with literacy: enrollment, teacher qualification, mother education, internet access
- Negative correlations with literacy: poverty rate, pupil-teacher ratio, travel time

### 4.3 State-Level Analysis

In [ ]:
from src.viz.plots import plot_state_bars

# State rankings
plot_state_bars(
    df,
    metric='literacy_rate',
    top_n=10,
    output_path='reports/figures/state_literacy_rankings.png'
)

Image('reports/figures/state_literacy_rankings.png')

### 4.4 Disparities Analysis

#### Gender Disparities

In [ ]:
from src.viz.plots import plot_group_comparison

# Literacy by sex
plot_group_comparison(
    df,
    metric='literacy_rate',
    group_by=['sex'],
    output_path='reports/figures/literacy_by_sex.png',
    title='Literacy Rate by Sex'
)

Image('reports/figures/literacy_by_sex.png')

#### Urban-Rural Disparities

In [ ]:
# Literacy by location
plot_group_comparison(
    df,
    metric='literacy_rate',
    group_by=['location'],
    output_path='reports/figures/literacy_by_location.png',
    title='Literacy Rate by Location'
)

Image('reports/figures/literacy_by_location.png')

In [ ]:
# Detailed breakdown by sex and location
print("Literacy Rate by Sex and Location:")
breakdown = df.groupby(['sex', 'location'])['literacy_outcome'].mean() * 100
print(breakdown.round(1))

# Visualize
fig, ax = plt.subplots(figsize=(8, 6))
breakdown.unstack().plot(kind='bar', ax=ax, color=['brown', 'green'], alpha=0.7)
ax.set_ylabel('Literacy Rate (%)', fontweight='bold')
ax.set_xlabel('Sex', fontweight='bold')
ax.set_title('Literacy Rate by Sex and Location', fontsize=14, fontweight='bold')
ax.legend(title='Location', labels=['Rural', 'Urban'])
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('reports/figures/literacy_sex_location.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Saved: reports/figures/literacy_sex_location.png")

### 4.5 Key Feature Relationships

In [ ]:
# Enrollment vs Literacy
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot by location
for location in df['location'].unique():
    subset = df[df['location'] == location]
    axes[0].scatter(subset['enrollment_rate'], subset['literacy_outcome'], 
                    alpha=0.3, label=location.capitalize(), s=1)

axes[0].set_xlabel('Enrollment Rate (%)', fontweight='bold')
axes[0].set_ylabel('Literacy Outcome (0/1)', fontweight='bold')
axes[0].set_title('Enrollment Rate vs Literacy', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Poverty vs Literacy
for location in df['location'].unique():
    subset = df[df['location'] == location]
    axes[1].scatter(subset['household_poverty_rate'], subset['literacy_outcome'], 
                    alpha=0.3, label=location.capitalize(), s=1)

axes[1].set_xlabel('Household Poverty Rate (%)', fontweight='bold')
axes[1].set_ylabel('Literacy Outcome (0/1)', fontweight='bold')
axes[1].set_title('Poverty Rate vs Literacy', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('reports/figures/key_relationships.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: reports/figures/key_relationships.png")

## 5. Logistic Regression Modeling

We'll fit logistic regression to identify which factors are most strongly associated with literacy outcomes.

**Important:** These models show associations, not causal effects. The synthetic data is designed to have realistic patterns, but real-world relationships are more complex.

In [ ]:
from src.modeling.logistic_regression import prepare_data, fit_statsmodels_logit, get_odds_ratios_table, fit_sklearn_logistic

# Prepare data
print("Preparing data for modeling...")
X_train, X_test, y_train, y_test, scaler, features = prepare_data(df, test_size=0.2)
print(f"Training samples: {len(X_train):,}")
print(f"Test samples: {len(X_test):,}")

In [ ]:
# Fit statsmodels Logit for detailed statistics
print("Fitting logistic regression (statsmodels)...")
sm_result = fit_statsmodels_logit(X_train, y_train)

# Get odds ratios
or_table = get_odds_ratios_table(sm_result)
print("\nOdds Ratios with 95% Confidence Intervals:")
print("=" * 80)
display(or_table.iloc[1:].round(3))  # Exclude intercept

### Interpretation of Key Findings

Let's translate the odds ratios into plain language:

In [ ]:
from src.modeling.logistic_regression import interpret_odds_ratio

print("Key Associations (Plain Language):")
print("=" * 80)

# Get significant features
significant = or_table[or_table['p_value'] < 0.05].iloc[1:]  # Exclude intercept

for feature in significant.index:
    or_val = significant.loc[feature, 'odds_ratio']
    interpretation = interpret_odds_ratio(feature, or_val)
    print(f"\n• {interpretation}")

In [ ]:
# Visualize odds ratios
from src.viz.plots import feature_effect_plot

feature_effect_plot(
    or_table,
    output_path='reports/figures/odds_ratios_plot.png',
    title='Feature Effects on Literacy (Odds Ratios with 95% CI)'
)

Image('reports/figures/odds_ratios_plot.png')

### Model Performance

In [ ]:
# Fit sklearn model for performance metrics
print("Fitting logistic regression (scikit-learn)...")
sk_model, metrics = fit_sklearn_logistic(X_train, y_train, X_test, y_test)

print("\nModel Performance:")
print("=" * 40)
print(f"Training Accuracy: {metrics['train_accuracy']:.3f}")
print(f"Test Accuracy: {metrics['test_accuracy']:.3f}")
print(f"Test ROC-AUC: {metrics['test_roc_auc']:.3f}")

print("\nConfusion Matrix (Test Set):")
cm = metrics['confusion_matrix']
print(f"  True Negatives:  {cm[0][0]:>6}")
print(f"  False Positives: {cm[0][1]:>6}")
print(f"  False Negatives: {cm[1][0]:>6}")
print(f"  True Positives:  {cm[1][1]:>6}")

## 6. Optional: Random Forest Model

Let's fit a simple tree-based model for comparison and to check feature importance.

In [ ]:
# Fit Random Forest
print("Fitting Random Forest classifier...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

# Predictions
rf_train_pred = rf_model.predict(X_train)
rf_test_pred = rf_model.predict(X_test)
rf_test_proba = rf_model.predict_proba(X_test)[:, 1]

print("\nRandom Forest Performance:")
print(f"  Training Accuracy: {accuracy_score(y_train, rf_train_pred):.3f}")
print(f"  Test Accuracy: {accuracy_score(y_test, rf_test_pred):.3f}")
print(f"  Test ROC-AUC: {roc_auc_score(y_test, rf_test_proba):.3f}")

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance (Random Forest):")
print(feature_importance.to_string(index=False))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(range(len(feature_importance)), feature_importance['importance'], alpha=0.7)
ax.set_yticks(range(len(feature_importance)))
ax.set_yticklabels([f.replace('_', ' ').title() for f in feature_importance['feature']])
ax.invert_yaxis()
ax.set_xlabel('Importance', fontweight='bold')
ax.set_title('Feature Importance (Random Forest)', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('reports/figures/feature_importance_rf.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Saved: reports/figures/feature_importance_rf.png")

**Note on Model Limitations:**
- These models show associations, not causal effects
- Synthetic data follows programmed patterns that may not reflect real-world complexity
- Real policy decisions require actual data, qualitative research, and domain expertise
- Omitted variable bias, measurement error, and other issues not addressed here

## 7. Save Summary Results

Let's save a summary for the policy brief.

In [ ]:
# Create summary for policy brief
summary = {
    "overall_statistics": {
        "overall_literacy_rate": float(df['literacy_outcome'].mean() * 100),
        "urban_literacy_rate": float(df[df['location']=='urban']['literacy_outcome'].mean() * 100),
        "rural_literacy_rate": float(df[df['location']=='rural']['literacy_outcome'].mean() * 100),
        "male_literacy_rate": float(df[df['sex']=='M']['literacy_outcome'].mean() * 100),
        "female_literacy_rate": float(df[df['sex']=='F']['literacy_outcome'].mean() * 100)
    },
    "model_performance": {
        "test_accuracy": float(metrics['test_accuracy']),
        "test_roc_auc": float(metrics['test_roc_auc'])
    },
    "key_associations": []
}

# Add top associations
for feature in significant.index[:5]:
    or_val = significant.loc[feature, 'odds_ratio']
    summary['key_associations'].append({
        'feature': feature,
        'odds_ratio': float(or_val),
        'interpretation': interpret_odds_ratio(feature, or_val)
    })

# Save
with open('reports/analysis_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ Saved analysis summary to reports/analysis_summary.json")

## 8. Summary and Conclusions

### Key Findings:

1. **Significant Disparities Exist:**
   - Urban areas have higher literacy rates than rural areas
   - Gender gaps may exist depending on the synthetic data pattern
   - State-level variation is substantial

2. **Positive Associations with Literacy:**
   - Higher enrollment rates
   - Better teacher qualifications
   - Higher mother education levels
   - Better internet and electricity access
   - Textbook availability

3. **Negative Associations with Literacy:**
   - Higher poverty rates
   - Higher pupil-teacher ratios (overcrowded classrooms)
   - Longer travel times to school

### Recommendations:

1. **Focus on Enrollment:** Strengthen programs that keep children in school
2. **Reduce Class Sizes:** Hire more qualified teachers, especially in underserved areas
3. **Address Poverty:** Conditional cash transfers, school feeding programs
4. **Improve Infrastructure:** Textbooks, electricity, internet access
5. **Target Rural Areas:** Mobile schools, community learning centers
6. **Mother Education:** Adult literacy programs have spillover effects on children

### Limitations:

- **Synthetic Data:** Results are illustrative only
- **Observational:** Cannot establish causation
- **Simplified Model:** Real world is more complex
- **Requires Validation:** Actual data and domain expertise needed

### Next Steps:

1. Collect real data from NBS, UBEC, and other sources
2. Conduct qualitative research (interviews, focus groups)
3. Pilot interventions with rigorous evaluation
4. Partner with education experts and policymakers
5. Continuously monitor and evaluate programs

In [ ]:
print("="*80)
print("Analysis Complete!")
print("="*80)
print("\nAll figures saved to: reports/figures/")
print("Summary saved to: reports/analysis_summary.json")
print("\nNext: View the Streamlit dashboard with 'streamlit run dashboards/app.py'")